In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('ab_browser_test.csv')

In [3]:
data.head()

,userID,browser,slot,n_clicks,n_queries,n_nonclk_queries
0,1,Browser #2,exp,23,32,19
1,3,Browser #4,exp,3,4,2
2,5,Browser #4,exp,29,35,16
3,6,Browser #4,control,12,6,0
4,7,Browser #4,exp,54,68,30


В данном задании вам нужно будет

проанализировать АБ тест, проведенный на реальных пользователях Яндекса
подтвердить или опровергнуть наличие изменений в пользовательском поведении между контрольной (control) и тестовой (exp) группами
определить характер этих изменений и практическую значимость вводимого изменения
понять, какая из пользовательских групп более всего проигрывает / выигрывает от тестируемого изменения (локализовать изменение)
Описание данных:

userID: уникальный идентификатор пользователя

browser: браузер, который использовал userID

slot: в каком статусе пользователь участвовал в исследовании (exp = видел измененную страницу, control = видел неизменную страницу)

n_clicks: количество кликов, которые пользоваль совершил за n_queries

n_queries: количество запросов, который совершил userID, пользуясь браузером browser

n_nonclk_queries: количество запросов пользователя, в которых им не было совершено ни одного клика

Обращаем ваше внимание, что не все люди используют только один браузер, поэтому в столбце userID есть повторяющиеся идентификаторы. В предлагаемых данных уникальным является сочетание userID и browser.

Основная метрика, на которой мы сосредоточимся в этой работе, — это количество пользовательских кликов на web-странице в зависимости от тестируемого изменения этой страницы.

Посчитайте, насколько в группе exp больше пользовательских кликов по сравнению с группой control в процентах от числа кликов в контрольной группе.

Полученный процент округлите до третьего знака после точки.

In [7]:
exp = data[data.slot == 'exp']
control = data[data.slot == 'control']

In [25]:
'%.5f' % ((exp.n_clicks.sum() / control.n_clicks.sum()))

'1.01614'

In [8]:
exp.n_clicks.sum()

3261823

In [9]:
control.n_clicks.sum()

3210027

Давайте попробуем посмотреть более внимательно на разницу между двумя группами (control и exp) относительно количества пользовательских кликов.

Для этого постройте с помощью бутстрепа 95% доверительный интервал для средних значений и медиан количества кликов в каждой из двух групп. Отметьте все верные утверждения.

In [4]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha=0.05):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [14]:
exp_median = [np.median(i) for i in get_bootstrap_samples(exp.n_clicks.values, 500)]
control_median = [np.median(i) for i in get_bootstrap_samples(control.n_clicks.values, 500)]

In [18]:
delta_medians = [i-j for i,j in zip(exp_median, control_median)]

In [19]:
print("95% confidence interval for the exp group:",  stat_intervals(exp_median))
print("95% confidence interval for the control group:",  stat_intervals(control_median))
print("95% confidence interval for the delta medians:",  stat_intervals(delta_medians))

95% confidence interval for the exp group: [ 5.  5.]
95% confidence interval for the control group: [ 4.  4.]
95% confidence interval for the delta medians: [ 1.  1.]


In [20]:
exp_mean = [np.mean(i) for i in get_bootstrap_samples(exp.n_clicks.values, 500)]
control_mean = [np.mean(i) for i in get_bootstrap_samples(control.n_clicks.values, 500)]
delta_means = [i-j for i,j in zip(exp_mean, control_mean)]
print("95% confidence interval for the exp group:",  stat_intervals(exp_mean))
print("95% confidence interval for the control group:",  stat_intervals(control_mean))
print("95% confidence interval for the delta medians:",  stat_intervals(delta_means))

95% confidence interval for the exp group: [ 11.50996333  11.65339024]
95% confidence interval for the control group: [ 11.20710805  11.35797731]
95% confidence interval for the delta medians: [ 0.2021061   0.40323723]
